## Quantum Genetic Algorithms
#### Awnit Singh Marta and Harshal Singh

Setting up IBM Q

In [2]:
from qiskit import QuantumProgram
import Qconfig
import os
import sys

from qiskit import QuantumProgram
from qiskit.tools.visualization import plot_histogram


from qiskit.tools.visualization import plot_histogram, plot_state

import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from IPython.display import Image

Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config['url']) # set the APIToken and API url

Check availibility

In [3]:
Q_program.available_backends()

['ibmqx_hpc_qasm_simulator',
 'ibmqx4',
 'ibmqx5',
 'ibmqx2',
 'ibmqx_qasm_simulator',
 'local_qasm_simulator',
 'local_clifford_simulator',
 'local_qiskit_simulator',
 'local_unitary_simulator']

In [4]:
Q_program.get_backend_status('ibmqx_qasm_simulator')

{'available': True, 'backend': 'ibmqx_qasm_simulator', 'pending_jobs': 0}

Define properties of the quantum gene
    
    • These variables were considered after reading through the papers referenced on our proposal
 
Some other variables, such as |0> and |1> have also been initialised here

In [9]:
p_size = 50
# Population size
chrom_len = 4
# Chromosome length
max_gen = int(input("Enter max length of generations: "))
# Currently, by letting ourself enter a number for the maximum
# we will be able to notice the difference
top_bottom = 3

actualpop_size=p_size+1
gen_len=chrom_len+1

alphabeta = Qobj()
probability = Qobj()
fitness = Qobj()

alphabeta.dims = [[1],[top_bottom]]
probability.dims = [[1],[p_size]]
fitness.dims = [[1],[p_size]]

ket0 = basis(2,0)
ket1 = basis(2,1)
# Hadamard gate
sqrt1_2 = 1/np.sqrt(2)
H = np.array([[sqrt1_2,sqrt1_2],[sqrt1_2,-sqrt1_2]])

# Quantum Chromosome
qpv = np.empty([actualpop_size, gen_len, top_bottom])
nqpv = np.empty([actualpop_size, gen_len, top_bottom])

# Classic Chromosome
chromosome = np.empty([actualpop_size, gen_len],dtype=np.int) 
child1 = np.empty([actualpop_size, gen_len, top_bottom])
child2 = np.empty([actualpop_size, gen_len, top_bottom])

best_chrom = np.empty([max_gen])

theta=0
iteration=0
bestchrom=0
generation=0

Enter max length of generations: 6


In [28]:
def initialpop():
    # Hadamard gate
    sqrt1_2 = 1/np.sqrt(2)
    H = np.array([[sqrt1_2,sqrt1_2],[sqrt1_2,-sqrt1_2]])
    # Rotation Q-gate
    theta = 0;
    rot = np.empty([2,2])
    N = 2 # 2-qubits
    qc_ex = QubitCircuit(N) # Initialize quantum circuit with 2 qubits
    print(qc_ex)
    for i in range(1,p_size):
        for j in range(1,chrom_len):
            theta=np.random.uniform(0,np.pi)
            
            qc_ex.add_gate("PHASEGATE", 0,None, theta) #random theta
            qc_ex.add_gate("SNOT", 0)   #hadamard


            qpv[i,j,0]=np.around(2*pow(alphabeta[0],2),2) 
            print(qpv[i,j,0])
        # beta squared

            qpv[i,j,1]=np.around(2*pow(alphabeta[1],2),2) 

In [29]:
# SHOW QUANTUM POPULATION                               #

#########################################################

def Show_population():

    i=1; j=1;

    for i in range(1,actualpop_size):

        print()

        print()

        print("qpv = ",i," : ")

        print()

        for j in range(1,gen_len):

            print(qpv[i, j, 0],end="")

            print(" ",end="")

        print()

        for j in range(1,gen_len):

            print(qpv[i, j, 1],end="")

            print(" ",end="")

    print()

In [30]:

# MAKE A MEASURE                                        #

#########################################################

# p_alpha: probability of finding qubit in alpha state

def Measure(p_alpha):

    for i in range(1,actualpop_size):

        print()

        for j in range(1,gen_len):

            if p_alpha<=qpv[i, j, 0]:

                chromosome[i,j]=0

            else:

                chromosome[i,j]=1

            print(chromosome[i,j]," ",end="")

        print()

    print()

In [31]:
# FITNESS EVALUATION                                    #

#########################################################

def Fitness_evaluation(generation):

    i=1; j=1; fitness_total=0; sum_sqr=0;

    fitness_average=0; variance=0;

    for i in range(1,actualpop_size):

        fitness[i]=0


In [32]:
# Define your problem in this section. For instance:    #

#                                                       #

# Let f(x)=abs(x-5/2+sin(x)) be a function that takes   #

# values in the range 0<=x<=15. Within this range f(x)  #

# has a maximum value at x=11 (binary is equal to 1011) #

#########################################################

    for i in range(1,actualpop_size):

        x=0;

        for j in range(1,gen_len):

            x=x+chromosome[i,j]*pow(2,gen_len-j-1)

           # replaces the value of x in the function f(x)

            y= np.fabs((x-5)/(2+np.sin(x)))

           # the fitness value is calculated below:

           # (Note that in this example is multiplied

           # by a scale value, e.g. 100)

            fitness[i]=y*100

#########################################################

      

        print("fitness = ",i," ",fitness[i])

        fitness_total=fitness_total+fitness[i]

    fitness_average=fitness_total/p_size

    i=1;

    while i<=p_size:

        sum_sqr=sum_sqr+pow(fitness[i]-fitness_average,2)

        i=i+1

    variance=sum_sqr/p_size

    if variance<=1.0e-4:

        variance=0.0

    # Best chromosome selection

    bestchrom=0;

    fitness_max=fitness[1];

    for i in range(1,actualpop_size):

        if fitness[i]>=fitness_max:

            fitness_max=fitness[i]

            bestchrom=i

    best_chrom[generation]=bestchrom

    # Statistical output

    f = open("output.dat","a")

    f.write(str(generation)+" "+str(fitness_average)+"\n")

    f.write(" \n")

    f.close()

    print("Population size = ",actualpop_size - 1)

    print("mean fitness = ",fitness_average)

    print("variance = ",variance," Std. deviation = ",np.sqrt(variance))

    print("fitness max = ",best_chrom[generation])

    print("fitness sum = ",fitness_total)



IndentationError: unexpected indent (<ipython-input-32-9e9ce2a4d5f7>, line 13)

In [33]:
# QUANTUM ROTATION GATE                                 #

#########################################################

def rotation():

    rot=np.empty([2,2])

    # Lookup table of the rotation angle

    for i in range(1,actualpop_size):

        for j in range(1,gen_len):
            int_marker = int(best_chrom[generation])

            if fitness[i]<fitness[int_marker]:
                

                if chromosome[i,j]==0 and chromosome[int_marker,j]==1:

                   # Define the rotation angle: delta_theta (e.g. 0.0785398163)

                    delta_theta=0.0785398163

                    rot[0,0]=np.cos(delta_theta); rot[0,1]=-np.sin(delta_theta);

                    rot[1,0]=np.sin(delta_theta); rot[1,1]=np.cos(delta_theta);

                    nqpv[i,j,0]=(rot[0,0]*qpv[i,j,0])+(rot[0,1]*qpv[i,j,1])

                    nqpv[i,j,1]=(rot[1,0]*qpv[i,j,0])+(rot[1,1]*qpv[i,j,1])

                    qpv[i,j,0]=round(nqpv[i,j,0],2)

                    qpv[i,j,1]=round(1-nqpv[i,j,0],2)

                if chromosome[i,j]==1 and chromosome[int_marker,j]==0:

                   # Define the rotation angle: delta_theta (e.g. -0.0785398163)

                    delta_theta=-0.0785398163

                    rot[0,0]=np.cos(delta_theta); rot[0,1]=-np.sin(delta_theta);

                    rot[1,0]=np.sin(delta_theta); rot[1,1]=np.cos(delta_theta);

                    nqpv[i,j,0]=(rot[0,0]*qpv[i,j,0])+(rot[0,1]*qpv[i,j,1])

                    nqpv[i,j,1]=(rot[1,0]*qpv[i,j,0])+(rot[1,1]*qpv[i,j,1])

                    qpv[i,j,0]=round(nqpv[i,j,0],2)

                    qpv[i,j,1]=round(1-nqpv[i,j,0],2)

             # if chromosome[i,j]==1 and chromosome[best_chrom[generation],j]==1:


In [34]:
# X-PAULI QUANTUM MUTATION GATE                         #

#########################################################

# pop_mutation_rate: mutation rate in the population

# mutation_rate: probability of a mutation of a bit

def mutation(pop_mutation_rate, mutation_rate):

    uallele=0;

    for i in range(1,actualpop_size):

        up=np.random.random_integers(100)

        up=up/100

        if up<=pop_mutation_rate:

            for j in range(1,gen_len):

                ualelle=up=np.random.random_integers(100)

                ualelle=ualelle/100

                if uallele<=mutation_rate:

                    nqpv[i,j,0]=qpv[i,j,1]

                    nqpv[i,j,1]=qpv[i,j,0]

                else:

                    nqpv[i,j,0]=qpv[i,j,0]

                    nqpv[i,j,1]=qpv[i,j,1]

        else:

            for j in range(1,gen_len):

                nqpv[i,j,0]=qpv[i,j,0]

                nqpv[i,j,1]=qpv[i,j,1]

    for i in range(1,actualpop_size):

        for j in range(1,gen_len):

            qpv[i,j,0]=nqpv[i,j,0]

            qpv[i,j,1]=nqpv[i,j,1]




In [35]:

# PERFORMANCE GRAPH                                     #

#########################################################

# Read the Docs in http://matplotlib.org/1.4.1/index.html

def plot_Output():

    data = np.loadtxt('output.dat')

    # plot the first column as x, and second column as y

    x=data[:,0]

    y=data[:,1]

    plt.plot(x,y)

    plt.xlabel('Generation')

    plt.ylabel('Fitness average')

    plt.xlim(0.0, 550.0)

    plt.show()

In [36]:

#########################################################

# PERFORMANCE GRAPH                                     #

#########################################################

# Read the Docs in http://matplotlib.org/1.4.1/index.html

def plot_Output():

    data = np.loadtxt('output.dat')

    # plot the first column as x, and second column as y

    x=data[:,0]

    y=data[:,1]

    plt.plot(x,y)

    plt.xlabel('Generation')

    plt.ylabel('Fitness average')

    plt.xlim(0.0, 550.0)

    plt.show()



########################################################

#                                                      #

# MAIN PROGRAM                                         #

#                                                      #

########################################################

def Q_GA():

    generation=0;

    print("============== GENERATION: ",generation," =========================== ")

    print()

    initialpop()

    Show_population()

    Measure(0.5)

    Fitness_evaluation(generation)

    while (generation<max_gen-1):

        print("The best of generation [",generation,"] ", best_chrom[generation])

        print()

        print("============== GENERATION: ",generation+1," =========================== ")

        print()

        rotation()

        mutation(0.01,0.001)

        generation=generation+1

        Measure(0.5)

        Fitness_evaluation(generation)

Q_GA()


plot_Output()



============== GENERATION:  0  =========================== 



ValueError: setting an array element with a sequence.